## Download Data ??

In [2]:
!wget -q https://www.dropbox.com/s/vs6ocyvpzzncvwh/new_articles.zip

In [3]:
!unzip -q new_articles.zip -d new_articles

## Load Enviro. Vars ??

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ['HUGGINGFACE_API'] = os.getenv('HUGGINGFACE_API')

## Import Necassary Library ??

In [3]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFaceHub
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader

## Load Data ??

In [5]:
loader = DirectoryLoader(
    "new_articles/",
    glob="./05-04*.txt",
    loader_cls=TextLoader,
    loader_kwargs={"encoding": "utf-8"}  # ✅ Prevent decode crash
)

In [6]:
document = loader.load()

document

[Document(metadata={'source': 'new_articles\\05-04-cma-generative-ai-review.txt'}, page_content='Well that was fast. The U.K.’s competition watchdog has announced an initial review of “AI foundational models”, such as the large language models (LLMs) which underpin OpenAI’s ChatGPT and Microsoft’s New Bing. Generative AI models which power AI art platforms such as OpenAI’s DALL-E or Midjourney will also likely fall in scope.\n\nThe Competition and Markets Authority (CMA) said its review will look at competition and consumer protection considerations in the development and use of AI foundational models — with the aim of understanding “how foundation models are developing and producing an assessment of the conditions and principles that will best guide the development of foundation models and their use in the future”.\n\nIt’s proposing to publish the review in “early September”, with a deadline of June 2 for interested stakeholders to submit responses to inform its work.\n\n“Foundation m

## Handle the Idea of Chunk Size and Chunk Overlap ??

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)

chunks = text_splitter.split_documents(document)

chunks

[Document(metadata={'source': 'new_articles\\05-04-cma-generative-ai-review.txt'}, page_content='Well that was fast. The U.K.’s competition watchdog has announced an initial review of “AI'),
 Document(metadata={'source': 'new_articles\\05-04-cma-generative-ai-review.txt'}, page_content='of “AI foundational models”, such as the large language models (LLMs) which underpin OpenAI’s'),
 Document(metadata={'source': 'new_articles\\05-04-cma-generative-ai-review.txt'}, page_content='OpenAI’s ChatGPT and Microsoft’s New Bing. Generative AI models which power AI art platforms such'),
 Document(metadata={'source': 'new_articles\\05-04-cma-generative-ai-review.txt'}, page_content='such as OpenAI’s DALL-E or Midjourney will also likely fall in scope.'),
 Document(metadata={'source': 'new_articles\\05-04-cma-generative-ai-review.txt'}, page_content='The Competition and Markets Authority (CMA) said its review will look at competition and consumer'),
 Document(metadata={'source': 'new_articles\\05-0

In [9]:
len(chunks) # Number of Chunks Created ??

407

In [10]:
chunks[1]

Document(metadata={'source': 'new_articles\\05-04-cma-generative-ai-review.txt'}, page_content='of “AI foundational models”, such as the large language models (LLMs) which underpin OpenAI’s')

## Creating VectorDB -> ChromaDB ??

In [12]:
from langchain import embeddings

persist_directory = "db"

embedding = HuggingFaceEmbeddings()

embedding

C:\Users\hp\AppData\Local\Temp\ipykernel_13704\167798456.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings()
C:\Users\hp\AppData\Local\Temp\ipykernel_13704\167798456.py:5: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding = HuggingFaceEmbeddings()


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-mpnet-base-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [ ]:
vectordb = Chroma.from_documents(documents=chunks, embedding=embedding, persist_directory=persist_directory)

In [ ]:
# Persiste The DB To Disk 

vectordb.persist()
vectordb = None

In [ ]:
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

## Make a Retriever ??

In [ ]:
retriever = vectordb.as_retriever()

In [ ]:
docs = retriever.get_relevant_documents("How much money did Microsoft raise?")

len(docs)

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [ ]:
retriever.search_type

In [ ]:
retriever.search_kwargs

In [ ]:
docs = retriever.get_relevant_documents("How much money did Microsoft raise?")

len(docs)

## Make a Chain ??

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=HuggingFaceHub,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [ ]:
query = "How mush money did Microsoft raise?"

llm_response = qa_chain(query)

llm_response

In [ ]:
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')

    for source in llm_response['source_documents']:
        print(source.metadata['source'])

In [ ]:
process_llm_response(llm_response)

## Deleting The DB ??

In [ ]:
!zip -r db.zip ./db

In [ ]:
# To Cleanup, You Can Delete the Collection ??
vectordb.delete_collection()
vectordb.persist()

# Delete the Directory ??
!rm -rf db/

## If You Want to Start Again To Use Your DB ??

In [ ]:
!unzip db.zip